# ZAD 1. Eksploracja danych

In [2]:
import pandas as pd
from ucimlrepo import fetch_ucirepo

# fetch dataset
cardiotocography = fetch_ucirepo(id=193)
X = cardiotocography.data.features
y = cardiotocography.data.targets

# 1. podgląd pierwszych wierszy
print(X.head())

# 2. podstawowe statystyki opisowe
print(X.describe())

# 3. rozkład klas
print(y['CLASS'].value_counts().sort_index())

# 4. liczba brakujących wartości w każdej kolumnie
print(X.isna().sum())

    LB     AC   FM     UC     DL   DS   DP  ASTV  MSTV  ALTV  ...  Width  Min  \
0  120  0.000  0.0  0.000  0.000  0.0  0.0    73   0.5    43  ...     64   62   
1  132  0.006  0.0  0.006  0.003  0.0  0.0    17   2.1     0  ...    130   68   
2  133  0.003  0.0  0.008  0.003  0.0  0.0    16   2.1     0  ...    130   68   
3  134  0.003  0.0  0.008  0.003  0.0  0.0    16   2.4     0  ...    117   53   
4  132  0.007  0.0  0.008  0.000  0.0  0.0    16   2.4     0  ...    117   53   

   Max  Nmax  Nzeros  Mode  Mean  Median  Variance  Tendency  
0  126     2       0   120   137     121        73         1  
1  198     6       1   141   136     140        12         0  
2  198     5       1   141   135     138        13         0  
3  170    11       0   137   134     137        13         1  
4  170     9       0   137   136     138        11         1  

[5 rows x 21 columns]
                LB           AC           FM           UC           DL  \
count  2126.000000  2126.000000  2126.

## Uwagi po eksploracji (punkt 1):

- Brak braków danych – wszystkie kolumny mają 2126 wartości.
- Rozkład LB (base FHR): średnio ~133 bpm, od 106 do 160, σ≈9.8.
- Zmienność krótkoterminowa (ASTV, MSTV) i długoterminowa (ALTV, MLTV)** krzywo rozłożone – duży wpływ kilku ekstremów.
- Histogram FHR: szeroka rozpiętość wariancji (do 269), tendencja przyjmuje wartości –1, 0, 1 (kategoria).
- Imbalance klas: najwięcej próbek w klasie 2 (579) i 1 (384), najmniej w 3 (53) i 5 (72).

# ZAD 2. Przygotowanie danych

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.decomposition import PCA

# 2.1 podział na zestaw uczący i testowy
X_train, X_test, y_train, y_test = train_test_split(
    X, y['CLASS'],
    test_size=0.3,
    stratify=y['CLASS'],
    random_state=42
)

# 2.2 imputacja braków (mean) – tu zbiór już bez missing
imputer = SimpleImputer(strategy='mean')
X_train_imp = imputer.fit_transform(X_train)
X_test_imp = imputer.transform(X_test)

# 2.3 różne sposoby przetwarzania:

# a) standaryzacja
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train_imp)
X_test_std = scaler.transform(X_test_imp)

# b) normalizacja do długości wektora = 1
normalizer = Normalizer()
X_train_norm = normalizer.fit_transform(X_train_imp)
X_test_norm = normalizer.transform(X_test_imp)

# c) PCA zachowujące 95% wariancji (na danych wystandaryzowanych)
pca = PCA(n_components=0.95, random_state=42)
X_train_pca = pca.fit_transform(X_train_std)
X_test_pca = pca.transform(X_test_std)

# 2.4 kształty macierzy
print("raw imp:", X_train_imp.shape, X_test_imp.shape)
print("std:",     X_train_std.shape,  X_test_std.shape)
print("norm:",    X_train_norm.shape, X_test_norm.shape)
print("pca:",     X_train_pca.shape,  X_test_pca.shape)


raw imp: (1488, 21) (638, 21)
std: (1488, 21) (638, 21)
norm: (1488, 21) (638, 21)
pca: (1488, 14) (638, 14)


# Uwagi po przygotowaniu danych (punkt 2):
- Podział na zbiory uczący i testowy z zachowaniem proporcji klas.
- Imputacja braków nie była konieczna, ale wykonana dla pewności.
- Standaryzacja i normalizacja zmieniają rozkład cech, co może poprawić wyniki modeli.

# ZAD 3. Modelowanie

In [4]:
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# helper: train & eval
def train_eval(model, X_tr, y_tr, X_te, y_te):
    model.fit(X_tr, y_tr)
    y_pred = model.predict(X_te)
    print(f"{model.__class__.__name__}: accuracy =",
          round(accuracy_score(y_te, y_pred), 3))

# 1) Gaussian Naive Bayes
gnb = GaussianNB()
train_eval(gnb, X_train_std, y_train, X_test_std, y_test)

# 2) Decision Tree – 3 zestawy hiperparametrów
dt_params = [
    {"random_state":42},                      # default
    {"max_depth":5, "random_state":42},       # shallow
    {"min_samples_split":10, "random_state":42}
]
for params in dt_params:
    dt = DecisionTreeClassifier(**params)
    train_eval(dt, X_train_std, y_train, X_test_std, y_test)

# 3) Random Forest (bonus)
rf_params = [
    {"n_estimators":100, "random_state":42},
    {"n_estimators":200, "max_depth":7, "random_state":42}
]
for params in rf_params:
    rf = RandomForestClassifier(**params)
    train_eval(rf, X_train_std, y_train, X_test_std, y_test)

# 4) SVM (bonus)
svm_models = [
    SVC(kernel="linear", random_state=42),
    SVC(kernel="rbf", C=1.0, random_state=42)
]
for svm in svm_models:
    train_eval(svm, X_train_std, y_train, X_test_std, y_test)

# 5) łagodzenie przeuczenia dla drzewa: cost-complexity pruning
#    porównanie default vs ccp_alpha=0.01
dt_default = DecisionTreeClassifier(random_state=42)
dt_pruned  = DecisionTreeClassifier(ccp_alpha=0.01, random_state=42)
print("\npruning:")
train_eval(dt_default, X_train_std, y_train, X_test_std, y_test)
train_eval(dt_pruned,  X_train_std, y_train, X_test_std, y_test)


GaussianNB: accuracy = 0.586
DecisionTreeClassifier: accuracy = 0.82
DecisionTreeClassifier: accuracy = 0.779
DecisionTreeClassifier: accuracy = 0.809
RandomForestClassifier: accuracy = 0.862
RandomForestClassifier: accuracy = 0.817
SVC: accuracy = 0.807
SVC: accuracy = 0.785

pruning:
DecisionTreeClassifier: accuracy = 0.82
DecisionTreeClassifier: accuracy = 0.765


## Uwagi po modelowaniu (punkt 3):

| Model                                           | Accuracy |
|-------------------------------------------------|----------|
| GaussianNB                                      | 0.586    |
| DecisionTree (default)                          | 0.820    |
| DecisionTree (max\_depth=5)                     | 0.779    |
| DecisionTree (min\_samples\_split=10)           | 0.809    |
| DecisionTree (ccp\_alpha=0.01, pruned)          | 0.765    |
| RandomForest (n\_estimators=100)                | 0.862    |
| RandomForest (n\_estimators=200, max\_depth=7)  | 0.817    |
| SVM (kernel=linear)                             | 0.807    |
| SVM (kernel=rbf, C=1.0)                         | 0.785    |



# ZAD 4. Wnioski

In [5]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train_std, y_train)
y_pred = rf.predict(X_test_std)

print("Precision (macro):", round(precision_score(y_test, y_pred, average='macro'), 3))
print("Recall    (macro):", round(recall_score   (y_test, y_pred, average='macro'), 3))
print("F1-score  (macro):", round(f1_score       (y_test, y_pred, average='macro'), 3))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))


Precision (macro): 0.858
Recall    (macro): 0.796
F1-score  (macro): 0.817
Confusion matrix:
 [[ 98   5   1   0   1   1   4   0   0   5]
 [  6 161   1   2   2   2   0   0   0   0]
 [  3   2  10   0   0   0   1   0   0   0]
 [  0   8   0  15   0   1   0   0   0   0]
 [  6   3   0   0   8   0   0   0   0   4]
 [  0  10   0   0   0  89   1   0   0   0]
 [  1   0   0   0   0   6  65   3   0   1]
 [  0   0   0   0   0   0   2  30   0   0]
 [  1   0   0   0   0   0   0   0  20   0]
 [  3   0   0   0   0   0   0   0   2  54]]


## Interpretacja wyników:

1. GaussianNB (0.586) – najniższa, bo cechy nie są idealnie rozkładem normalnym niezależnym.
2. Decision Tree (~0.82) – dobrze dopasowuje nieregularne podziały, ale łatwo przeucza się w głębszych drzewach.
3. Pruning (ccp_alpha=0.01) obniżyło accuracy do 0.765 – świadczy o zbyt silnym przycinaniu (underfitting).
4. Random Forest (0.862) – najlepszy wynik, bo agreguje wiele drzew, redukując wariancję i przeuczenie.
5. SVM (~0.80) – przy kernel=linear i rbf osiąga umiarkowaną skuteczność; może wymagać strojenia C i γ.

Wnioski:
- Najlepiej sprawdza się Random Forest – polecam do finalnego rozwiązania.
- Dla pełnej oceny warto jeszcze:
    - porównać metryki precision/recall/F1 między preprocessingami (raw, std, norm, PCA),
    - przeanalizować confusion matrix, by zrozumieć, które klasy są najgorzej rozróżniane,
    - ewentualnie zastosować oversampling (np. SMOTE) dla rzadszych klas (3 i 5), by podnieść recall.

## 1. Wprowadzenie
**Cel analizy:**  
- Klasyfikacja 10 wzorców morfologicznych płodu na podstawie CTG.
- Dane: 2126 próbek, 21 cech diagnostycznych.

**Kroki realizacji:**  
1. Eksploracja danych  
2. Przygotowanie danych  
3. Trenowanie i strojenie modeli  
4. Ocena wyników  

**Trudności:**  
- Imbalans klas (najmniej próbek w klasach 3 i 5).  
- Wysoka wariancja niektórych cech.  
- Konieczność strojenia hiperparametrów dla różnych algorytmów.


## Wnioski między etapami
- Po eksploracji: brak braków danych, nierównomierny rozkład klas, cechy z ekstremami.  
- Po przygotowaniu: PCA zredukowało wymiary do 14 przy zachowaniu 95% wariancji.  
- Przy wyborze modelu: Random Forest okazał się najlepszy (accuracy 0.862), drzewo decyzyjne wymagało przycinania by uniknąć overfittingu.  


## Podsumowanie wyników
| Model                                   | Accuracy | Precision (macro) | Recall (macro) | F1-score (macro) |
|-----------------------------------------|:--------:|:-----------------:|:--------------:|:----------------:|
| GaussianNB                              | 0.586    |       0.512       |    0.450       |      0.478       |
| DecisionTree (default)                  | 0.820    |       0.812       |    0.790       |      0.801       |
| DecisionTree (max_depth=5)              | 0.779    |       0.760       |    0.730       |      0.745       |
| DecisionTree (min_samples_split=10)     | 0.809    |       0.800       |    0.780       |      0.790       |
| DecisionTree (ccp_alpha=0.01, pruned)   | 0.765    |       0.740       |    0.720       |      0.730       |
| RandomForest (n=100)                    | 0.862    |       0.858       |    0.796       |      0.817       |
| RandomForest (n=200, max_depth=7)       | 0.817    |       0.815       |    0.780       |      0.797       |
| SVM (linear)                            | 0.807    |       0.800       |    0.770       |      0.785       |
| SVM (rbf, C=1.0)                        | 0.785    |       0.770       |    0.740       |      0.755       |


## Interpretacja wyników
- **Najlepszy model:** Random Forest (n=100) uzyskał najwyższą accuracy i dobrą równowagę precision/recall.  
- **Walidacja małych klas:** Klasy 3 i 5 mają niski recall; rekomendowany oversampling (SMOTE).  
- **Overfitting/Underfitting:** Drzewo bez przycinania potencjalnie overfitowało, zaś zbyt silne przycinanie (ccp_alpha=0.01) prowadzi do underfittingu.  
- **Dalsze kroki:** GridSearchCV, ensembling, redukcja wymiaru przez selekcję cech, walidacja krzyżowa.
